In [1]:
import pandas as pd
import os
from datetime import date

BASE_DIR = os.getcwd()
INVENTORY_FILE = os.path.join(BASE_DIR, "inventory.xlsx")
PICKUP_FILE = os.path.join(BASE_DIR, "pickup.xlsx")


def get_latest_inventory():
    df = pd.read_excel(INVENTORY_FILE)
    product_cols = df.select_dtypes(include="number").columns
    return df.iloc[-1][product_cols].astype(float)


def get_order_master():
    df = pd.read_excel(PICKUP_FILE, index_col=0)
    threshold = df.loc["しきい値"]
    add_qty = df.loc["追加量"]
    return threshold, add_qty


def calculate_orders(latest, threshold, add_qty):
    orders = {}
    for p in latest.index:
        if p in threshold.index and p in add_qty.index:
            if pd.notna(latest[p]) and latest[p] < threshold[p]:
                orders[p] = int(add_qty[p])
    return orders


def create_order_mail(order_items):
    today = date.today().strftime("%Y年%m月%d日")
    subject = f"野菜発注のお願い（{today}）"

    body_lines = [f"・{p}：{q}" for p, q in order_items.items()]
    body = "\n".join(body_lines)

    mail_body = f"""株式会社〇〇
ご担当者様

いつもお世話になっております。
下記内容にて発注をお願いいたします。

【発注内容】
{body}

以上、よろしくお願いいたします。
"""

    return subject, mail_body


# ===== 実行 =====
latest = get_latest_inventory()
threshold, add_qty = get_order_master()
orders = calculate_orders(latest, threshold, add_qty)

if not orders:
    print("✅ 発注なし")
else:
    subject, body = create_order_mail(orders)
    print("【件名】")
    print(subject)
    print("\n【本文】")
    print(body)


✅ 発注なし
